In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
#Loading the data with pandas
data = pd.read_csv('states.csv')
#Select only the 'date' column from whole dataset
date = data['date']
total = data['total']

In [6]:
#Encoding the categorical data(the state)
state = data[['date','state']]

state_table = []
state_names = []
new_state = []

j = 0
for i in state['state']:
    if i not in state_names:
        state_names.append(i)         #In this array i am appending only the state names but only for once
        state_table.append([i,j])     #If i am appending the state name for the first time i am building the table 
        j = j + 1                     # Incrementing j
for i in state['state']:
    for j in state_table: 
        if i == j[0]:
            new_state.append(j[1])    # Creating new array for only numerical values 
            
state = state.drop(columns=['state']) # dropping the old text array
state['state'] = new_state            # adding the new numerical array in the same place

,date,state
0,20200325,0
1,20200325,1
2,20200325,2
3,20200325,3
4,20200325,4
...,...,...
1088,20200304,40
1089,20200304,44
1090,20200304,47
1091,20200304,52


In [7]:
#Functions for transformation
def state_scale(state, inverse): #Just simple minmax scale, i am taking the max value from the numerical global array 
    Max = max(new_state)
    if inverse == True:
        arr = []
        for i in state:
            arr.append(i*Max)
        
    elif inverse == False:
        arr = []
        for i in state:
            arr.append(i/Max)
        
    return arr

def y_scale(y, Max, inverse): #Simple minmax scale, i am taking the max value from te numerical global target array 
    if inverse == True:
        arr = []
        for i in y:
            arr.append(i*Max)
        
    elif inverse == False:
        arr = []
        for i in y:
            if i == 0:
                arr.append(0)
            else:
                arr.append(i/Max)
        
    return arr

In [8]:
#Completing the missing values (NaN)
datad = data.drop(columns=['state','date'])#dropping values who does not have NaN vals
imputer = SimpleImputer(missing_values=np.NaN, strategy='median')#In place of NaN adding mean value
transformed_values = imputer.fit_transform(datad.values)#Fitting the transformation
result = pd.DataFrame(transformed_values)
result.columns = datad.columns
#Adding the column names from the original dataset
result['state'] = data['state']
result['date'] = data['date']
result

,positive,negative,pending,hospitalized,death,totalTestResults,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,total,state,date
0,42.0,1649.0,20.0,1.0,1.0,1691.0,1.0,1.0,663.0,6.0,669.0,1691.0,AK,20200325
1,283.0,2529.0,20.0,33.0,0.0,2812.0,0.0,0.0,423.0,68.0,491.0,2812.0,AL,20200325
2,280.0,1437.0,0.0,22.0,2.0,1717.0,2.0,0.0,490.0,62.0,552.0,1717.0,AR,20200325
3,0.0,222.0,20.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AS,20200325
4,450.0,323.0,53.0,8.0,6.0,773.0,1.0,0.0,10.0,93.0,103.0,826.0,AZ,20200325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,3.0,29.0,18.0,33.0,2.0,32.0,0.0,0.0,22.0,5.0,36.0,50.0,OR,20200304
1089,0.0,5.0,0.0,33.0,2.0,5.0,0.0,0.0,22.0,5.0,36.0,5.0,SC,20200304
1090,1.0,222.0,20.0,33.0,2.0,1.0,0.0,0.0,22.0,5.0,36.0,1.0,TX,20200304
1091,39.0,222.0,20.0,33.0,2.0,39.0,0.0,0.0,22.0,5.0,36.0,39.0,WA,20200304


In [6]:
#This is for one day in the future.
def one_day_forecast(state, result):
    if state not in state_names:
        print("State name is not valid")
    else:
        final = result.loc[result['state'] == state]#finding all records with STATE from the dataset
        final = final.drop(columns=['state'])
        final = final.sort_values(by=['date']).reset_index(drop = True)
        total = final['total']
        max_total = max(total)

        scalerX = StandardScaler().fit(final)#Fitting the scaler
        cval = scalerX.transform(final)
        
        #Adding rolling window. Generating new training dataset. Window contains X values, target is y.
        window_size = 1#One day...
        window = [] 
        target = []

        for j in range(len(cval)-window_size):
            arr = []
            for i in range(window_size):
                arr.extend(cval[j+i])
            window.append(arr)
            target.append(total[j+i+1])

        target = y_scale(target, max_total, False)

        last_date = final['date'].max()
        one_day_predict = cval[-1]#This is the last day from the dataset in the state. We are predicting the value for the next day but training the model on last known day(last day in datasaet).
        parameters = {'C':[0.001,0.01,0.1,0.5,1.5,2,2.5,3,3.5],'epsilon':[0.1,0.01,0.001,0.0001,0.00001]}
        svr = SVR()
        clf_gs = GridSearchCV(svr, parameters)
        clf_gs.fit(window, target)#fitting the grid search
        best_params = clf_gs.best_params_   
        clf = SVR(C = best_params['C'], epsilon = best_params['epsilon'])#Epsilon and C are hyperparameters from SVR algorithm. Try to learn the matemathics in background to understand how it works.
        clf.fit(window, target)#fitting the model

        svr_pred = clf.predict(one_day_predict.reshape(1, -1))#resheped because it's only one array.
        
        print(f'State: {state}')
        print(f'Day: {last_date+1}')
        print(f'Total = {y_scale(svr_pred, max_total, True)}')

one_day_forecast('AK',result)

State: AK
Day: 20200326
Total = [723.5314061142307]


In [ ]:
#This is for last know day. I am doing this in order to calculate the accuracy.
def last_day(state, result):
    if state not in state_names:
        print("State name is not valid")
    else:
        final = result.loc[result['state'] == state]#finding all records with STATE from the dataset
        final = final.drop(columns=['state'])
        final = final.sort_values(by=['date']).reset_index(drop = True)
        total = final['total']
        max_total = max(total)

        scalerX = StandardScaler().fit(final)#Fitting the scaler
        cval = scalerX.transform(final)
        
        #Adding rolling window. Generating new training dataset. Window contains X values, target is y.
        window_size = 1#One day...
        window = [] 
        target = []

        for j in range(len(cval)-window_size):
            arr = []
            for i in range(window_size):
                arr.extend(cval[j+i])
            window.append(arr)
            target.append(total[j+i+1])

        target = y_scale(target, max_total, False)

        last_date = final['date'].max()
        one_day_predict = cval[-2]#This is the last day from the dataset in the state. We are predicting the value for the next day but training the model on last known day(last day in datasaet).
        parameters = {'C':[0.001,1,10,50,5,100,25,75,150,200,250,230],'epsilon':[0.1,0.01,0.001,0.0001,0.00001]}

        svr = SVR()
        clf_gs = GridSearchCV(svr, parameters)
        clf_gs.fit(window, target)#fitting the grid search
        best_params = clf_gs.best_params_   
        clf = SVR(C = best_params['C'], epsilon = best_params['epsilon'])#Epsilon and C are hyperparameters from SVR algorithm. Try to learn the matemathics in background to understand how it works.
        clf.fit(window, target)#fitting the model

        svr_pred = clf.predict(one_day_predict.reshape(1, -1))#resheped because it's only one array.
        
        return total.values[-1], y_scale(svr_pred, max_total, True)


arr = []
for state in state_names:
    pred, test = last_day(state,result)
    if pred == 0:
        arr.append(0)
    else:
        arr.append(test/pred)

/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/py

/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/py

/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/py

/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/alongrod/opt/anaconda3/envs/tensorflow/lib/py

In [8]:
print(f'Score: {float(sum(arr)/len(arr))*100} %')

Score: 75.72318479006614 %
